In [1]:
from datetime import *
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
import pandas as pd
import json
import os
import yahoo_fin.stock_info as si
import yfinance as yf
from utils.utils_bigquery import *

In [2]:
# Ruta credenciales de CGP
key_path = key_path

# Datos de origen de datos en Bigquery
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_info'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
# Traemos los ticker del SP500, Nasdaq y Russell2000
unique_tickers = bigquery.run_query(
    f"""
        SELECT DISTINCT ticker FROM (
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_sp500_tickers
        UNION ALL
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_russell_2000_tickers
        UNION ALL
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_nasdaq_tickers
        ) AS ticker_combined
    """
)

unique_tickers

,ticker
0,COP
1,EOG
2,OXY
3,HES
4,FANG
...,...
7128,ARTV
7129,CHRO
7130,KYTX
7131,MGX


In [5]:
# En el caso de que los ticker varien, hacemos un incremental para traernos los nuevos que aun no estan en bbdd
current_df = bigquery.run_query(
            f"""
            SELECT
                ticker
            FROM {table_conca}
            """
            )
df_incremental = unique_tickers[~unique_tickers['ticker'].isin(current_df['ticker'])]
df_incremental

,ticker
2603,AKO/A
2604,AKO/B
2648,BF/A
2649,BF/B
2654,BIO/B
2663,BRK/A
2664,BRK/B
2698,CRD/A
2699,CRD/B
2779,HEI/A


In [8]:
import yfinance as yf
import pandas as pd

data = []

for i, row in df_incremental.iterrows():
    ticker = row['ticker']
    
    try:
        # Obtener datos históricos
        ticker_data = yf.Ticker(ticker).info
        
        # Convertir el diccionario en una fila de DataFrame
        data_row = pd.Series(ticker_data)
        
        # Agregar el símbolo como una columna en los datos históricos
        data_row['ticker'] = ticker
        
        # Añadir los datos a la lista
        data.append(data_row)
    
    except Exception as e:
        print(f"Error procesando {ticker}: {e}")
        # Continúa con el siguiente ticker si ocurre un error
        continue

# Concatenar todas las filas y convertir en un DataFrame
df = pd.DataFrame(data)
df.head()

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKO/A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AKO%2FA&crumb=gwu53Teopj8


Error procesando AKO/A: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKO/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AKO%2FB&crumb=gwu53Teopj8


Error procesando AKO/B: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BF/A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BF%2FA&crumb=gwu53Teopj8


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HVT/A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HVT%2FA&crumb=gwu53Teopj8


Error procesando HVT/A: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WSO/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WSO%2FB&crumb=gwu53Teopj8


Error procesando WSO/B: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}
Error procesando None: 'NoneType' object has no attribute 'upper'


""


In [9]:
# Identificar columnas de tipo 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Convertir las columnas de tipo 'object' a listas
for col in object_columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

In [10]:
# Lista de columnas a eliminar
columns_to_drop = ['forwardPE', 'priceToSalesTrailing12Months', 'industrySymbol']
columns_to_drop = []

# Eliminar las columnas
df = df.drop(columns=columns_to_drop)

In [24]:
# Convertimos a tipo string
df["trailingPE"] = df["trailingPE"].astype(str)

In [61]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='append', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
